In [1]:
import s3fs


fs = s3fs.S3FileSystem(anon=True)


directory_path = 'aws-public-blockchain/v1.0/eth/blocks'


files = fs.ls(directory_path)


parquet_files = [file for file in files]

print(parquet_files[:5])


['aws-public-blockchain/v1.0/eth/blocks/date=2015-07-30', 'aws-public-blockchain/v1.0/eth/blocks/date=2015-07-31', 'aws-public-blockchain/v1.0/eth/blocks/date=2015-08-01', 'aws-public-blockchain/v1.0/eth/blocks/date=2015-08-02', 'aws-public-blockchain/v1.0/eth/blocks/date=2015-08-03']


In [2]:
import pandas as pd
import s3fs

# Setup access to the S3 bucket with s3fs
fs = s3fs.S3FileSystem(anon=True)

# Specify the directory path in the S3 bucket where the .parquet files are located
directory_path = 'aws-public-blockchain/v1.0/eth/blocks'

# List all files in the specified directory of the S3 bucket
#directory_path = 's3://aws-public-blockchain/v1.0/eth/token_transfers'

# Ensure you're getting the correct full paths for the .parquet files
files = fs.ls(directory_path)
parquet_files = [f's3://{file}' for file in files if file.endswith('.parquet')]

# Initialize an empty list to store DataFrames loaded from each .parquet file
dfs = []

# Loop through the top 5 .parquet file paths
for file_path in parquet_files[:5]:
    # Use s3fs to open and read each .parquet file into a pandas DataFrame
    with fs.open(file_path, mode='rb') as f:
        df = pd.read_parquet(f, engine='pyarrow')
        # Append the loaded DataFrame to the list
        dfs.append(df)

# Concatenate all the DataFrames in the list into a single DataFrame
# This combines the data from the top 5 .parquet files into one dataset
combined_df = pd.concat(dfs, ignore_index=True)

# Display the head of the combined DataFrame to verify its contents
print(combined_df.head())


ValueError: No objects to concatenate

In [8]:
import pandas as pd
import s3fs


fs = s3fs.S3FileSystem(anon=True)


directory_path = 'aws-public-blockchain/v1.0/eth/blocks/date=2020-01-01'


files = fs.ls(directory_path)



parquet_files = [file for file in files if file.endswith('.parquet')]


file_path = 's3://' + parquet_files[0]

with fs.open(file_path, mode='rb') as f:
    df = pd.read_parquet(f, engine='pyarrow')


print(df.columns.tolist())


['difficulty', 'hash', 'miner', 'nonce', 'number', 'size', 'timestamp', 'total_difficulty', 'base_fee_per_gas', 'gas_limit', 'gas_used', 'extra_data', 'logs_bloom', 'parent_hash', 'state_root', 'receipts_root', 'transactions_root', 'sha3_uncles', 'transaction_count', 'date', 'last_modified']


In [7]:
import pandas as pd
import s3fs


fs = s3fs.S3FileSystem(anon=True)


directory_path = 's3://aws-public-blockchain/v1.0/eth/contracts/date=2015-08-10'


files = fs.ls(directory_path)



parquet_files = [file for file in files if file.endswith('.parquet')]


file_path = 's3://' + parquet_files[0]

with fs.open(file_path, mode='rb') as f:
    df = pd.read_parquet(f, engine='pyarrow')


print(df.head())


                                      address  \
0  0xed5868f9ec0651f5a9797fad4716c1269b776730   
1  0x5fe5b7546d1628f7348b023a0393de1fc825a4fd   
2  0x02aeac9ca07bc56f37d5ec61b28a3dffd7cff0a3   
3  0x90ab39213f1d947e2a3b8936ac37dca2556a9ece   
4  0xa18d7182d5935578958a07fd7e9d062ddd418761   

                                            bytecode  block_number  \
0  0x60606040523615610053576000357c01000000000000...         65407   
1  0x606060405236156100615760e060020a60003504631c...         66126   
2  0x606060405236156100775760e060020a600035046321...         66156   
3  0x60606040526000357c01000000000000000000000000...         65327   
4  0x60606040526000357c01000000000000000000000000...         66173   

                                          block_hash     block_timestamp  \
0  0x0b14125ff9fdb36bef8a566c1a3cf6aa1356a366b2f4... 2015-08-10 19:53:14   
1  0x80cb4a1f7d35cad112c1f035a5f31ccf52b450227fa6... 2015-08-10 23:26:20   
2  0x4823b4a52e889d9bb4860fba9ee41b69907dbe1d703f... 201

In [27]:
import pandas as pd
import s3fs
import sqlite3

# Setup access to the S3 bucket
fs = s3fs.S3FileSystem(anon=True)

# Specify the directory path in the S3 bucket
directory_path = 's3://aws-public-blockchain/v1.0/eth/contracts/date=2015-08-10'
files = fs.ls(directory_path)

# Filter for .parquet files
parquet_files = [file for file in files if file.endswith('.parquet')]

# Assuming there's at least one .parquet file, get the path to the first one
file_path = 's3://' + parquet_files[0]
print(parquet_files[:5])  # Print the first 5 parquet files found

# Read the .parquet file into a pandas DataFrame
with fs.open(file_path, mode='rb') as f:
    df = pd.read_parquet(f, engine='pyarrow')

# Print the first few rows of the DataFrame
print(df.head())

# Use a context manager to handle the SQLite database connection
with sqlite3.connect('contracts.db') as conn:
    # Create table with primary key, including 'NOT NULL' for clarity
    conn.execute('''
    CREATE TABLE IF NOT EXISTS con (
        id INTEGER PRIMARY KEY NOT NULL,  
        address TEXT,
        bytecode TEXT,
        block_number INTEGER,
        block_hash TEXT,
        block_timestamp TEXT,
        date TEXT,
        last_modified TEXT
    );
    ''')

    # Write the DataFrame to the SQLite table, considering 'if_exists' strategy.
    df.to_sql('con', conn, if_exists='append', index=False)

print(f"Data from {file_path} has been successfully loaded into the 'con' table in 'contracts.db'.")


['aws-public-blockchain/v1.0/eth/contracts/date=2015-08-10/part-00000-3759dbc2-5841-4cd8-b13c-e9776aa78129-c000.snappy.parquet']
                                      address  \
0  0xed5868f9ec0651f5a9797fad4716c1269b776730   
1  0x5fe5b7546d1628f7348b023a0393de1fc825a4fd   
2  0x02aeac9ca07bc56f37d5ec61b28a3dffd7cff0a3   
3  0x90ab39213f1d947e2a3b8936ac37dca2556a9ece   
4  0xa18d7182d5935578958a07fd7e9d062ddd418761   

                                            bytecode  block_number  \
0  0x60606040523615610053576000357c01000000000000...         65407   
1  0x606060405236156100615760e060020a60003504631c...         66126   
2  0x606060405236156100775760e060020a600035046321...         66156   
3  0x60606040526000357c01000000000000000000000000...         65327   
4  0x60606040526000357c01000000000000000000000000...         66173   

                                          block_hash     block_timestamp  \
0  0x0b14125ff9fdb36bef8a566c1a3cf6aa1356a366b2f4... 2015-08-10 19:53:14   
1  

In [16]:
import s3fs

# Setup access to the S3 bucket
fs = s3fs.S3FileSystem(anon=True)

# Base directory path in the S3 bucket
base_directory_path = 'aws-public-blockchain/v1.0/eth/token_transfers'

# Get the list of date directories
date_directories = fs.ls(base_directory_path)

# Filter date directories to include only those from 2015
date_directories_2015 = [
    dir_path for dir_path in date_directories 
    if 'date=2016-01' in dir_path or 'date=2016-02' in dir_path or 'date=2016-03' in dir_path
]
# Assuming you want to load .parquet files from the top 20 date directories in 2015
top_date_directories_2015 = date_directories_2015

# Display the filtered date directories
print(top_date_directories_2015)

['aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-06', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-07', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-09', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-12', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-13', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-14', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-15', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-17', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-18', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-19', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-20', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-21', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-22', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-23', 'aws-public-blockchain/v1.0/eth/token_transfers/date=2016-01-24', 'aws-publ